# Testning av KNN modell.

In [209]:
import os
import numpy as np
from skimage import io, transform
from skimage.color import rgb2gray

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [210]:
root_folder = "C:/Users/emmel/Desktop/sick_fishes/dataset/SB-FishDisease" #emmelies root

In [211]:
def load_dataset(root_folder):
    sick_fish_list = []
    healthy_fish_list = []

    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                
                # Check if the current item is a file (not a subdirectory)
                if os.path.isfile(file_path):
                    img = io.imread(file_path)
                    label = file_name.split('_')[0].lower()

                    if "healthy" in label:
                        healthy_fish_list.append(img)
                    else:
                        sick_fish_list.append(img)

    return sick_fish_list, healthy_fish_list

In [212]:
sick_fish_list, healthy_fish_list = load_dataset(root_folder)

In [ ]:
# print(len(sick_fish_list))
# print(len(healthy_fish_list))

In [214]:
#sick_fish_list[1]

In [215]:
def resize_images(image_list, target_size=(100, 100)):
    resized_images = [transform.resize(img, target_size, mode='reflect', anti_aliasing=True) for img in image_list]
    return resized_images

def convert_to_grayscale(image_list):
    grayscale_images = [rgb2gray(img) for img in image_list]
    return grayscale_images

def normalize_images(image_list):
    normalized_images = [img / 255.0 for img in image_list]
    return normalized_images

def flatten_and_resize_images(image_list, target_size=(100, 100), expected_size=30000):
    resized_and_flattened_images = [
        img if img.size == expected_size else None
        for img in [
            transform.resize(img, target_size, mode='reflect', anti_aliasing=True)
            for img in image_list
        ]
    ]
    resized_and_flattened_images = [img.flatten() for img in resized_and_flattened_images if img is not None]
    return resized_and_flattened_images

In [216]:
resized_sick_fish_list = resize_images(sick_fish_list)
resized_healthy_fish_list = resize_images(healthy_fish_list)

In [218]:
normalized_sick_fish_list = normalize_images(resized_sick_fish_list)
normalized_healthy_fish_list = normalize_images(resized_healthy_fish_list)

In [219]:
resized_and_flattened_sick_fish_list = flatten_and_resize_images(sick_fish_list)
resized_and_flattened_healthy_fish_list = flatten_and_resize_images(healthy_fish_list)

In [ ]:
#resized_and_flattened_sick_fish_list[2]

In [226]:
print(len(resized_and_flattened_sick_fish_list))
print(len(resized_and_flattened_healthy_fish_list))

267
154


In [221]:
x = resized_and_flattened_sick_fish_list + resized_and_flattened_healthy_fish_list
y = ["sick"] * len(resized_and_flattened_sick_fish_list) + ["healthy"] * len(resized_and_flattened_healthy_fish_list)

In [222]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [223]:
knn_model = KNeighborsClassifier(n_neighbors=5)
x_train_flat = [img.flatten() for img in x_train]
knn_model.fit(x_train_flat, y_train)

KNeighborsClassifier()

### Flattening

In [224]:
indices_to_flatten = [i for i, img in enumerate(x_test) if img.size == 30000]

x_test_flat_reshaped = np.array([img.flatten() for i, img in enumerate(x_test) if i in indices_to_flatten])

Shape of x_train_flat: (336, 30000)
Shape of x_test_flat_reshaped: (85, 30000)
Length of x_test: 85
Length of flattened x_test: 85
Indices to flatten: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84]


### Test printar vid fel på flattening:

In [225]:
# indices_not_flattened = [i for i in range(len(x_test)) if i not in indices_to_flatten]

# # Print the indices of images not flattened
# print("Indices not flattened:", indices_not_flattened)

Indices not flattened: []


In [205]:
# print("Size of image at index 78:", x_test[78].size)
# print("Size of image at index 84:", x_test[84].size)

Size of image at index 78: 40000
Size of image at index 84: 40000


# Predict:

In [227]:
y_pred = knn_model.predict(x_test_flat_reshaped)

### Fler felhanterings printar:

In [228]:
# print("Length of x:", len(x))
# print("Length of y:", len(y))

Length of x: 421
Length of y: 421


In [229]:
# print("Length of y_test:", len(y_test))
# print("Length of y_pred:", len(y_pred))

Length of y_test: 85
Length of y_pred: 85


# Accuracy:

In [230]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6705882352941176
